This project is made for Reasoning Agents part where the fundamental idea is to apply Reinforcement Learning to Mario game.

## Libraries
We have included several libraries which will be needed for the project. They are Gym, pandas, numpy, tensorflow and others.

In [8]:
import tkinter as tk
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import gym
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY
from nes_py.wrappers import JoypadSpace
from IPython.display import clear_output

from keras.models import save_model, load_model
import time


## Environment

In [9]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, RIGHT_ONLY)

## Simple test
This is used to test the game and see how it exactly works.

In [11]:
total_reward = 0
done = True
for step in range(100000):
    env.render()
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    print(info)
    total_reward += reward
    clear_output(wait=True)
env.close()

{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 293, 'world': 1, 'x_pos': 594, 'y_pos': 85}


KeyboardInterrupt: 

## Class for the Mario

In [ ]:
class MarioAgent:
    def __init__(self, state_size, action_size):
        # we need to create variables for our agent here
        self.state_space = state_size
        self.action_space = action_size
        self.memory = deque(maxlen=5000)

        #giving the epsilon value for exploration and exploiation
        self.epsilon = 1
        self.max_epsilon = 1
        self.min_epsilon = 0.01
        self.decay_epsilon = 0.0001

        # we then need to build the NN for the agent
        self.main_network = self.build_network()
        self.target_network = self.build_network()
        self.update_target_network()

    def build_network(self):
        model = Sequential()
        model.add(Conv2D(64, (4,4), strides=4, padding='same',input_shape=self.state_space))
        model.add(Activation('relu'))

        model.add(Conv2D(64, (4,4), strides=2, padding='same'))
        model.add(Activation('relu'))

        model.add(Conv2D(64, (3,3), strides=1, padding='same'))
        model.add(Activation('relu'))
        model.add(Flatten())

        model.add(Dense(512, activation='relu'))
        model.add(Dense(256, activation='relu'))
        model.add(Dense(self.action_space, activation='linear'))

        model.compile(loss='mse', optimizer=Adam())

    def update_target_network(self):
        self.target_network.set_weights(self.main_network.get_weights())

    def act(self, state):
        if random.uniform(0,1) < self.epsilon:
            return np.random.randint(self.action_space)
        Q_value = self.main_network.predict(state)
        return np.argmax(Q_value[0])

    def update_epsilon(self, episode):
        self.epsilon = self.min_epsilon + (self.max_epsilon - self.min_epsilon)*np.exp(-self.decay_epsilon * episode)
